In [1]:
import torch
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import os, h5py

from utils import generate_detect_sg, generate_gt_sg
from sgg_benchmark.config.paths_catalog import DatasetCatalog

In [2]:
path = "/home/maelic/Documents/Scene-Graph-Benchmark-Cuda11.7/datasets/VG150/curated/VG-SGG-with-attri.h5"
# load with h5py
data = h5py.File(path, 'r')

In [4]:
print(data['attributes'])

<HDF5 dataset "attributes": shape (1441984, 10), type "<i8">


In [ ]:
file_path = '/home/maelic/Documents/PhD/Datasets/GQA/sceneGraphs/train_sceneGraphs.json'
gqa = json.load(open(file_path, 'r'))

In [ ]:
print(gqa['2386622'])

# Run Evaluation for test or train split

In [ ]:
BASE_PATH="/home/maelic/Documents/Scene-Graph-Benchmark-Cuda11.7/"
SGDET_DIR=BASE_PATH+"checkpoints/VG150/baseline/upload_causal_motif_sgdet"
FASTER_RCNN_DIR=BASE_PATH+"checkpoints/VG150/curated/pretrained_faster_rcnn/model_final.pth"
GLOVE_DIR="/home/maelic/glove"
EVAL_TYPE="test"
CONFIG="configs/VG150/curated/e2e_relation_X_101_32_8_FPN_1x.yaml"
DATASET="VG_stanford_filtered_with_attribute"
TASK="sgdet"

In [ ]:
stats = json.load(open('/home/maelic/Documents/PhD/MyModel/Scene-Graph-Benchmark-Cuda11.7/analysis/sg_statistics.json'))

In [ ]:
#!cd .. && pwd && CUDA_VISIBLE_DEVICES=0 python tools/relation_test_net.py --config-file $CONFIG --task  --verbose MODEL.ROI_RELATION_HEAD.PREDICTOR CausalAnalysisPredictor MODEL.ROI_RELATION_HEAD.CAUSAL.EFFECT_TYPE TDE MODEL.ROI_RELATION_HEAD.CAUSAL.FUSION_TYPE sum MODEL.ROI_RELATION_HEAD.CAUSAL.CONTEXT_LAYER motifs  SOLVER.IMS_PER_BATCH 32 TEST.IMS_PER_BATCH 1 DTYPE "float16" SOLVER.MAX_ITER 30000 SOLVER.VAL_PERIOD 2000 SOLVER.CHECKPOINT_PERIOD 2000 SOLVER.PRE_VAL False GLOVE_DIR $GLOVE_DIR MODEL.PRETRAINED_DETECTOR_CKPT $FASTER_RCNN_DIR OUTPUT_DIR $SGDET_DIR DATASETS.TO_TEST $EVAL_TYPE

In [ ]:
!CUDA_VISIBLE_DEVICES=0 python tools/relation_test_net.py --config-file "configs/VG150/curated/e2e_relation_X_101_32_8_FPN_1x.yaml" --task "sgdet" --verbose MODEL.ROI_RELATION_HEAD.PREDICTOR CausalAnalysisPredictor MODEL.ROI_RELATION_HEAD.CAUSAL.EFFECT_TYPE TDE MODEL.ROI_RELATION_HEAD.CAUSAL.FUSION_TYPE sum MODEL.ROI_RELATION_HEAD.CAUSAL.CONTEXT_LAYER vctree SOLVER.IMS_PER_BATCH 32 TEST.IMS_PER_BATCH 12 DTYPE "float16" SOLVER.MAX_ITER 30000 SOLVER.VAL_PERIOD 2000 SOLVER.CHECKPOINT_PERIOD 2000 SOLVER.PRE_VAL False GLOVE_DIR /home/maelic/glove MODEL.PRETRAINED_DETECTOR_CKPT /home/maelic/Documents/Scene-Graph-Benchmark-Cuda11.7/checkpoints/VG150/curated/pretrained_faster_rcnn/model_final.pth OUTPUT_DIR /home/maelic/Documents/Scene-Graph-Benchmark-Cuda11.7/checkpoints/VG150/curated/sgdet_vctree_tde DATASETS.TO_TEST train

In [3]:
result_path = SGDET_DIR + '/inference/' + DATASET +'_' + EVAL_TYPE + '/' 
info_dict = json.load(open(result_path + 'visual_info.json'))
detected_result = torch.load(result_path+'eval_results.pytorch')
result_dict = torch.load(result_path + 'result_dict.pytorch')
vg_dict = json.load(open(DatasetCatalog.DATASETS[DATASET]['dict_file']))
vg_data = h5py.File(DatasetCatalog.DATASETS[DATASET]['roidb_file'], 'r')

In [4]:
output_scene_graphs = generate_detect_sg(detected_result, info_dict, vg_dict, 0.3)
pred = [len(pred[0]['relations']) for i, pred in enumerate(output_scene_graphs.values())]
print("Number of total relations predicted: ", sum(pred))

Generating scene graphs : 100%|██████████| 5001/5001 [07:18<00:00, 11.41it/s]

Number of total relations predicted:  232672


In [ ]:
pred = [len(pred[0]['relations']) for i, pred in enumerate(output_scene_graphs.values())]
print(sum(pred))

In [12]:
input_scene_graphs = generate_gt_sg(vg_data, vg_dict, info_dict)
print(len(input_scene_graphs))
print(len([os.path.basename(img['img_file']).strip('.jpg') for img in info_dict]))

100%|██████████| 5001/5001 [00:00<00:00, 52449.64it/s]

3765
5001


## Compute frequency

In [13]:
def gt_triplets(input_scene_graphs):
    triplets = []
    for graph in tqdm(input_scene_graphs.values()):
        for r in graph[0]['relations']:
            rel = graph[0]['entities'][r[0]]+'-'+vg_dict['idx_to_predicate'][str(r[2])]+'-'+graph[0]['entities'][r[1]]
            triplets.append(rel)
    df = pd.DataFrame(triplets, columns =['Triplet'])
    # count triplet occurences
    df = df.groupby('Triplet').size().reset_index(name='Count')
    # sort by count
    df = df.sort_values(by=['Count'], ascending=False)
    return df
print("Ranking of triplets in the input scene graphs")
gt_freq = gt_triplets(input_scene_graphs)
print("Total number of triplets in the input scene graphs: ", gt_freq['Count'].sum())
gt_freq.head(10)
# total number of triplets


Ranking of triplets in the input scene graphs


100%|██████████| 3765/3765 [00:00<00:00, 316223.21it/s]

Total number of triplets in the input scene graphs:  25259


,Triplet,Count
3365,window-on-building,4597
579,car-on-street,797
426,building-has-window,758
1508,leaf-on-tree,469
1831,man-wearing-shirt,286
2216,pillow-on-bed,221
3370,window-on-house,206
1624,light-on-pole,202
2968,tire-on-car,201
239,book-on-shelf,200


In [10]:
def pred_triplets(output_scene_graphs):
    triplets = []
    for graph in tqdm(output_scene_graphs.values()):
        for r in graph[0]['relations']:
            rel = graph[0]['entities'][r[0]]+'-'+r[2]+'-'+graph[0]['entities'][r[1]]
            triplets.append(rel)
    df = pd.DataFrame(triplets, columns =['Triplet'])
    # count triplet occurences
    df = df.groupby('Triplet').size().reset_index(name='Count')
    # sort by count
    df = df.sort_values(by=['Count'], ascending=False)
    return df
print("Ranking of triplets in the input scene graphs")
out = pred_triplets(output_scene_graphs)
print("Total number of triplets in the input scene graphs: ", out['Count'].sum())
out.head(10)


Ranking of triplets in the input scene graphs


100%|██████████| 5001/5001 [00:00<00:00, 99468.48it/s]

Total number of triplets in the input scene graphs:  232672


,Triplet,Count
10967,man-wearing-shirt,3522
10906,man-watching-man,3320
15141,shirt-on-man,1674
20201,woman-wearing-shirt,1208
10963,man-wearing-pant,1139
14778,sheep-looking at-sheep,1132
10968,man-wearing-shoe,1000
20155,woman-watching-man,976
1555,bird-looking at-bird,974
10938,man-watching-woman,958


## Compute average triplets score

In [ ]:
obj_list = ['man', 'woman', 'person']
def triplets_ranking(output_scene_graphs):
    triplets_score = []
    for graph in tqdm(output_scene_graphs.values()):
        for r in graph[0]['relations']:
            rel = graph[0]['entities'][r[0]]+'-'+r[2]+'-'+graph[0]['entities'][r[1]]
            triplets_score.append([rel, r[3]])
    df = pd.DataFrame(triplets_score, columns =['Triplet', 'Score'])
    # average the dataframe by triplet
    df = df.groupby('Triplet')['Score'].mean().reset_index()
    # sort the dataframe by score
    df = df.sort_values(by=['Score'], ascending=False)
    # filter by triplet containing man or woman only
    df = df[df['Triplet'].str.contains('|'.join(obj_list))]
    return df
print(triplets_ranking(output_scene_graphs)[:50])


In [6]:
def triplets_ranking(output_scene_graphs):
    triplets_score = []
    for graph in tqdm(output_scene_graphs.values()):
        for r in graph[0]['relations']:
            rel = graph[0]['entities'][r[0]]+'-'+r[2]+'-'+graph[0]['entities'][r[1]]
            triplets_score.append([rel, r[4]])
    df = pd.DataFrame(triplets_score, columns =['Triplet', 'Rel_score'])
    # average the dataframe by triplet
    df = df.groupby('Triplet')['Rel_score'].mean().reset_index()
    # sort the dataframe by score
    df = df.sort_values(by=['Rel_score'], ascending=False)
    # filter by triplet containing man or woman only
    #df = df[df['Triplet'].str.contains('|'.join(obj_list))]
    return df
print(triplets_ranking(output_scene_graphs)[:50])

100%|██████████| 5001/5001 [00:00<00:00, 14905.88it/s]

                        Triplet  Rel_score
5479             drawer-in-room   0.968571
14445              room-has-leg   0.959393
4970            dog-eating-food   0.957489
4666               cup-in-glass   0.943067
12946            plane-has-nose   0.933713
19337       window-behind-child   0.925519
4891               desk-has-leg   0.923503
4772        curtain-behind-girl   0.922174
7787          hand-holding-coat   0.922117
7146          girl-riding-horse   0.912991
3129           bus-behind-horse   0.911485
5209          door-behind-child   0.910659
2777           boy-riding-horse   0.905584
19016         vegetable-in-bowl   0.901771
11194     mountain-behind-woman   0.901466
15625       sidewalk-under-face   0.901067
11152       mountain-behind-cow   0.894293
1083                bed-in-room   0.892868
12939            plane-has-door   0.888168
1811             boat-has-wheel   0.887122
11154  mountain-behind-elephant   0.886597
1678           board-behind-boy   0.882468
12605      